In [ ]:
# default_exp diff_analysis_manager

In [ ]:
#export
import alphaquant.diffquant_utils as aqutils
def run_pipeline_from_preconfigured_files(input_file, samplemap_file, results_dir = ".", minrep = 2, condpair_combinations = None,outlier_correction = True, median_offset = False, pre_normed_intensity_file = None, dia_fragment_selection = False, runtime_plots = False, volcano_fdr =0.05, volcano_fcthresh = 0.5, annotation_file = None):
    input_data = aqutils.import_data(input_file)
    samplemap_df = aqutils.load_samplemap(samplemap_file)
    input_processed, samplemap_df_processed = aqutils.prepare_loaded_tables(input_data, samplemap_df)
    run_pipeline(input_processed, samplemap_df, results_dir, condpair_combinations,minrep, outlier_correction, median_offset, pre_normed_intensity_file, dia_fragment_selection, runtime_plots, volcano_fdr, volcano_fcthresh, annotation_file)


In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np
import statsmodels.stats.multitest as mt
from time import time


def run_pipeline(unnormed_df, labelmap_df, results_dir = "./results",  condpair_combinations = None, minrep = 2, min_num_ions = 1, minpep = 1, cluster_threshold_pval = 1e-3, cluster_threshold_fcfc = 0.5, take_median_ion = True,outlier_correction = True, normalize = True, use_iontree_if_possible = True,
get_ion2clust = False, median_offset = False, pre_normed_intensity_file = None, dia_fragment_selection = False, runtime_plots = False, volcano_fdr =0.05, volcano_fcthresh = 0.5,annotation_file = None):
    """Run the differential analyses.
    """
    
    res_dfs = []
    pep_dfs = []
    pep2prot = dict(zip(unnormed_df.index, unnormed_df['protein']))

    if condpair_combinations == None:
        conds = labelmap_df["condition"].unique()
        condpair_combinations = combinations(conds, 2)
        

    for condpair in condpair_combinations:
        print(condpair)
        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        if (len(c1_samples.index)<2) | len(c2_samples.index)<2:
            print(f"condpair has not enough samples c1:{len(c1_samples)} c2: {len(c2_samples)}, skipping")
            continue
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=minrep, axis=0)
        if (len(df_c1.index)<5) | (len(df_c2.index)<5):
            print(f"condpair has not enough data for processing c1: {len(df_c1.index)} c2: {len(df_c2.index)}, skipping")
            continue
    
        res, peps = analyze_condpair(df_c1, df_c2, c1_samples, c2_samples, pep2prot,results_dir,condpair, minrep, min_num_ions, minpep,cluster_threshold_pval, cluster_threshold_fcfc, take_median_ion,outlier_correction, normalize, use_iontree_if_possible, get_ion2clust,median_offset, pre_normed_intensity_file , dia_fragment_selection, 
        runtime_plots, volcano_fdr, volcano_fcthresh, annotation_file)
        res_dfs.append(res)
        pep_dfs.append(peps)
    
    res_df = pd.concat(res_dfs)
    pep_df = pd.concat(pep_dfs)

    return res_df, pep_df



In [ ]:
#export
import alphaquant.background_distributions as aqbg
import alphaquant.diff_analysis as aqdiff
import alphaquant.normalization as aqnorm
import alphaquant.visualizations as aqviz
import alphaquant.diffquant_utils as aqutils
import alphaquant.cluster_ions as aqclust
import anytree


def analyze_condpair(df_c1, df_c2, c1_samples, c2_samples, pep2prot, results_dir,condpair, minrep, min_num_ions, minpep,cluster_threshold_pval, cluster_threshold_fcfc, take_median_ion,outlier_correction, normalize, use_iontree_if_possible, get_ion2clust,median_offset, pre_normed_intensity_file, dia_fragment_selection, runtime_plots, volcano_fdr, volcano_fcthresh, annotation_file):
    t_zero = time()
    print(f"start processeing condpair {condpair}")
    prot2diffions = {}
    p2z = {}
    ion2clust = {}
    prots = []
    iontrees = []
    pvals = []
    fdrs = []
    fcs = []
    consistency_scores = []
    numions = []
    peps = []
    pep_pvals = []
    pep_fdrs = []
    pep_fcs = []
    pep_prots = []
    peps_included = []
    pseudoint1 = []
    pseudoint2 = []
    condpairs = []

    if normalize:
        df_c1_normed, df_c2_normed = aqnorm.get_normalized_dfs(df_c1, df_c2, c1_samples, c2_samples, minrep, runtime_plots,pre_normed_intensity_file)#, "./test_data/normed_intensities.tsv")
    else:
        df_c1_normed = df_c1
        df_c2_normed = df_c2
    if results_dir != None:
        write_out_normed_df(df_c1_normed,df_c2_normed, pep2prot, results_dir, condpair)
    t_normalized = time()
    normed_c1 = aqbg.ConditionBackgrounds(df_c1_normed, p2z)
    #write_out_ion2nonan_ion2idx(normed_c1, "./test_data/", "c1")
    normed_c2 = aqbg.ConditionBackgrounds(df_c2_normed, p2z)
    #compare_context_boundaries_against_ref("./test_data/reference_context_boundaries_c2.tsv",normed_c2)
    #write_out_ion2nonan_ion2idx(normed_c2, "./test_data/", "c2")
    t_bgdist_fin = time()
    ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()
    use_ion_tree = list(ions_to_check)[0].startswith("SEQ_") & use_iontree_if_possible
    bgpair2diffDist = {}
    deedpair2doublediffdist = {}
    count_ions=0
    for ion in ions_to_check:
        t_ion = time()
        vals1 = normed_c1.ion2nonNanvals.get(ion)
        vals2 = normed_c2.ion2nonNanvals.get(ion)
        diffDist = aqbg.get_subtracted_bg(bgpair2diffDist,normed_c1, normed_c2,ion, p2z)
        t_subtract_end = time()
        diffIon = aqdiff.DifferentialIon(vals1, vals2, diffDist, ion, outlier_correction)
        t_diffion = time()
        protein = pep2prot.get(ion)
        prot_ions = prot2diffions.get(protein, list())
        prot_ions.append(diffIon)
        prot2diffions[protein] = prot_ions
        peps.append(ion)
        pep_pvals.append(diffIon.p_val)
        pep_fcs.append(diffIon.fc)
        pep_prots.append(protein)

        if count_ions%2000==0:
            print(f"checked {count_ions} of {len(ions_to_check)} ions")
        
        count_ions+=1

        t_iterfin = time()
        #print(f"t_init {t_subtract_start-t_ion} t_diffdist {t_subtract_end -t_subtract_start} t_diffion {t_iterfin - t_ion}")
    count_prots = 0
    for prot in prot2diffions.keys():
        ions = prot2diffions.get(prot)
        if len(ions)<min_num_ions:
            continue
        diffprot = aqdiff.DifferentialProtein(prot,ions, median_offset, dia_fragment_selection)
        if use_ion_tree:
            clustered_root_node = aqclust.get_scored_clusterselected_ions(prot, ions,normed_c1, normed_c2, bgpair2diffDist, p2z, deedpair2doublediffdist, pval_threshold_basis = cluster_threshold_pval, fcfc_threshold = cluster_threshold_fcfc, take_median_ion=take_median_ion)
            #print(anytree.RenderTree(clustered_root_node))
            #if not clustered_root_node.is_included:
             #   continue
            iontrees.append(clustered_root_node)
            pval, fc, consistency_score, ions_included = aqclust.get_diffresults_from_clust_root_node(clustered_root_node)
            num_peptides = len(anytree.findall(clustered_root_node, filter_ = lambda x : x.type == 'seq'))
            if num_peptides < minpep:
                continue

        else:
            pval, fc, consistency_score, ions_included = diffprot.pval, diffprot.fc, np.nan,diffprot.ions
        
        if get_ion2clust:
            ionclust_protein = aqclust.find_fold_change_clusters_base_ions([[x] for x in ions],normed_c1,normed_c2, bgpair2diffDist,p2z, deedpair2doublediffdist, fc_threshold=cluster_threshold_fcfc, pval_threshold_basis=cluster_threshold_pval)
            ion2clust.update({x.name:y for x,y in ionclust_protein.items()})
        prots.append(prot)
        pvals.append(pval)
        fcs.append(fc)
        consistency_scores.append(consistency_score)
        numions.append(len(ions_included))
        condpairs.append(aqutils.get_condpairname(condpair))
        peps_included.extend(ions_included)
        if count_prots%100==0:
            print(f"checked {count_prots} of {len(prot2diffions.keys())} prots")
        count_prots+=1
    
        pseudoint1_cond, pseudoint2_cond = aqdiff.calc_pseudo_intensities(ions, normed_c2, diffprot.fc)
        pseudoint1.append(pseudoint1_cond)
        pseudoint2.append(pseudoint2_cond)

    fdrs = mt.multipletests(pvals, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    pep_fdrs = mt.multipletests(pep_pvals, method='fdr_bh', is_sorted=False, returnsorted=False)[1]

    res_df = pd.DataFrame({'condpair' : condpairs,'protein' : prots, 'fdr' : fdrs, 'pval':pvals, 'log2fc' : fcs, 'consistency_score' : consistency_scores,'num_ions' : numions, 'pseudoint1' :  pseudoint1, 'pseudoint2' : pseudoint2})
    pep_df = pd.DataFrame({'condpair' : [aqutils.get_condpairname(condpair) for x in range(len(peps))], 'ion' : peps, 'protein' : pep_prots,'pval' : pep_pvals, 'log2fc' : pep_fcs})
    pep_df["fdr"] = pep_fdrs
   # pep_df = pep_df[pep_df["ion"].isin(peps_included)]

    if runtime_plots:
        aqviz.volcano_plot(res_df, significance_cutoff = volcano_fdr, log2fc_cutoff = volcano_fcthresh)
        aqviz.volcano_plot(pep_df,significance_cutoff = volcano_fdr, log2fc_cutoff = volcano_fcthresh)

    if results_dir!=None:
        if annotation_file != None: #additional annotations can be added before saving
            annot_df = pd.read_csv(annotation_file, sep = "\t")
            intersect_columns = annot_df.columns.intersection(pep_df.columns)
            if(len(intersect_columns)>0):
                print(list(intersect_columns))
                res_df = res_df.merge(annot_df, on=list(intersect_columns), how= 'left')
                pep_df = pep_df.merge(annot_df, on= list(intersect_columns), how = 'left')
        if use_ion_tree:
            aqclust.export_roots_to_json(iontrees,condpair,results_dir)
        if get_ion2clust:
            ion2clust_df = pd.DataFrame(ion2clust.items(), columns=['ion', 'cluster'])
            ion2clust_df.to_csv(f"{results_dir}/{aqutils.get_condpairname(condpair)}.ion2clust.tsv", sep = "\t", index=None)

        res_df.to_csv(f"{results_dir}/{aqutils.get_condpairname(condpair)}.results.tsv", sep = "\t", index=None)
        pep_df.to_csv(f"{results_dir}/{aqutils.get_condpairname(condpair)}.results.ions.tsv", sep = "\t", index=None)



    return res_df, pep_df
    

In [ ]:
#export
import numpy as np
import os
def write_out_normed_df(normed_df_1, normed_df_2, pep2prot, results_dir, condpair):
    merged_df = normed_df_1.merge(normed_df_2, left_index = True, right_index = True)
    merged_df = 2**merged_df
    merged_df = merged_df.replace(np.nan, 0)
    merged_df["protein"] = list(map(lambda x : pep2prot.get(x),merged_df.index))
    if not os.path.exists(f"{results_dir}/"):
        os.makedirs(f"{results_dir}/")
    merged_df.to_csv(f"{results_dir}/{aqutils.get_condpairname(condpair)}.normed.tsv", sep = "\t")

In [ ]:
def merge_normed_dfs(normed_df_1, normed_df_2, condpair):
    merged_df = normed_df_1.merge(normed_df_2, left_index = True, right_index = True)
    merged_df = 2**merged_df
    merged_df = merged_df.replace(np.nan, 0)
    merged_df["condpair"] = condpair
    return merged_df

In [ ]:
#export
import pandas as pd
import numpy as np

#read in proteomics datafiles, log the intensities
def read_tables(peptides_tsv, samplemap_tsv, pepheader = None, protheader = None):
    samplemap = pd.read_csv(samplemap_tsv, sep="\t")
    peps = pd.read_csv(peptides_tsv,sep="\t")

    if pepheader != None:
        peps = peps.rename(columns = {pepheader : "ion"})
    if protheader != None:
        peps = peps.rename(columns = {protheader: "protein"})
    peps = peps.set_index("ion")
    headers = ['protein'] + samplemap["sample"].to_list()

    for sample in samplemap["sample"]:
        peps[sample] = np.log2(peps[sample].replace(0, np.nan))
    return peps[headers], samplemap

In [ ]:
#hide
import pandas as pd

def write_out_ion2nonan_ion2idx(cb, outfolder, condname):
    
    ion2nan_conv = [[a, x.tolist()] for a, x in cb.ion2nonNanvals.items()]
    
    ion2nan_df = pd.DataFrame(ion2nan_conv)
    display(ion2nan_df)
    print(ion2nan_df.iloc[41775, 1])
    idx2ion_df = pd.DataFrame(list(cb.idx2ion.items()))

    ion2nan_df.to_csv(f"{outfolder}/ion2nonans_{condname}.tsv", sep = "\t", index = False)
    idx2ion_df.to_csv(f"{outfolder}/idx2ion_{condname}.tsv", sep = "\t", index = False)


    

In [ ]:
#hide
import pandas as pd
import numpy as np
def compare_context_boundaries_against_ref(ref_file, cond_bg):
    ref_bounds = pd.read_csv(ref_file, sep = "\t", names = ["l_ref", "u_ref"])
    ref_bounds["l"] = pd.Series(np.array(cond_bg.context_ranges).T[0])
    ref_bounds["u"] = pd.Series(np.array(cond_bg.context_ranges).T[1])
    ref_bounds["l-ref"] = (ref_bounds["l_ref"] / ref_bounds["l"]).abs()
    ref_bounds["u-ref"] = (ref_bounds["u_ref"] / ref_bounds["u"]).abs()
    display(ref_bounds)

In [ ]:
#hide

#protein_df, peptide_df = benchmark_proteomics("./test_data/peptides.txt", "./test_data/samples.map", "./test_data/prot2organism.tsv")
#protein_df.to_csv("./test_data/AP_protein_out.tsv", sep = "\t", index= False)
#peptide_df.to_csv("./test_data/AP_peptide_out.tsv", sep = "\t", index= False)


In [ ]:
# import sys
# sys.path.append('/Users/constantin/workspace/EmpiRe/nbdev/MS-EmpiRe_Python')
# from alphaquant.background_distributions import *
# from alphaquant.normalization import *
# from alphaquant.diff_analysis import *
# from alphaquant.visualizations import *
# from alphaquant.benchmarking import *
# from alphaquant.diffquant_utils import *